In [ ]:
################################################################################
# BELOW IS CODE ADAPTED FROM:
# 
# Escape Velocity Labs, 28/08/22, 'Advanced Reinforcement Learning in Python: from DQN to SAC', 
# Accessed at https://www.udemy.com/course/advanced-reinforcement/ 
#
# Note: Whilst the structure of this code is heavily taken from Escape Velocity Labs, there are also
# some self-coded elements involved in this adaptation.
################################################################################

In [ ]:
### Install dependencies
### Note: you need to restart the runtime after you run this cell

!apt-get install -y xvfb # Used to display videos and GUI's in this notebook

!pip install \
    gym==0.21 \
    gym[box2d] \
    pytorch-lightning==1.6.0 \
    optuna \
    pyvirtualdisplay

# gym -> Environment 
# gym[box2d] -> Dependency for box2d category in the gym
# pytorch-lightning -> Reinforcement learning and neural networks
# pyvirtualdisplay -> Used to display videos in the environment

In [ ]:
### Create a display to render the environment in google servers

from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

In [ ]:
### Import remaining packages and set up variables related to hardware

import copy
import gym
import torch
import random
import itertools
import matplotlib.pyplot as plt
import pandas as pd
import optuna

import numpy as np
import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import Tensor, nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningModule, Trainer

from pytorch_lightning.callbacks import EarlyStopping

from gym.wrappers import RecordVideo, RecordEpisodeStatistics, TimeLimit

from optuna.integration import PyTorchLightningPruningCallback

# Use GPU if there is one available
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

num_gpus = torch.cuda.device_count()

In [ ]:
### Create a function which takes episode number as input, and outputs a video of the environment of the given episode

def display_video(episode=0):
  video_file = open(f'/content/videos/rl-video-episode-{episode}.mp4', "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"<video width=600 controls><source src='{video_url}'></video>")

In [ ]:
### Create the actor network class

class Actor(nn.Module):

  def __init__(self, hidden_size, obs_size, out_dims, min, max, num_layers):
    super().__init__()
    self.min = torch.from_numpy(min).to(device)
    self.max = torch.from_numpy(max).to(device)
    if num_layers == 1:
      self.net = nn.Sequential(
          nn.Linear(obs_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, out_dims),
          nn.Tanh()
      )
    elif num_layers == 2:
      self.net = nn.Sequential(
          nn.Linear(obs_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, out_dims),
          nn.Tanh()
      )
    elif num_layers == 3:
      self.net = nn.Sequential(
          nn.Linear(obs_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, out_dims),
          nn.Tanh()
      )
    
  # This function takes a state as input and outputs the highest return action
  def mu(self, x):
    if isinstance(x, np.ndarray):
      x = torch.from_numpy(x).to(device)
    return self.net(x.float()) * self.max

  # This function takes a state as input, and returns the highest return action with clipped noise
  def forward(self, x, epsilon=0.0, noise_clip=None):
    mu = self.mu(x)
    noise = torch.normal(0, epsilon, mu.size(), device=mu.device)
    if noise_clip is not None:
      noise = torch.clamp(noise, -noise_clip, noise_clip)
    mu = mu + noise
    action = torch.max(torch.min(mu, self.max), self.min)
    action = action.detach().cpu().numpy() # Maybe not detach
    return action


In [ ]:
### Create the Q-Network class

class DQN(nn.Module):

  def __init__(self, hidden_size, obs_size, out_dims, num_layers):
    super().__init__()
    if num_layers == 1:
      self.net = nn.Sequential(
          nn.Linear(obs_size + out_dims, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),           
          nn.Linear(hidden_size, 1),
      )
    elif num_layers == 2:
      self.net = nn.Sequential(
          nn.Linear(obs_size + out_dims, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),
          nn.Linear(hidden_size, hidden_size),
          nn.ReLU(),                
          nn.Linear(hidden_size, 1),
      )
    elif num_layers == 3:
        self.net = nn.Sequential(
            nn.Linear(obs_size + out_dims, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),     
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),                
            nn.Linear(hidden_size, 1),
        )

  # This function takes a state-action pair and returns a Q-value (predicted expected return)
  def forward(self, state, action):
    if isinstance(state, np.ndarray):
      state = torch.from_numpy(state).to(device)
    if isinstance(action, np.ndarray):
      action = torch.from_numpy(action).to(device)
    in_vector = torch.hstack((state, action))
    return self.net(in_vector.float())


In [ ]:
### Create the ReplayBuffer class, from which we make out buffer

class ReplayBuffer:

  def __init__(self, capacity):
    self.buffer = deque(maxlen=capacity)
  
  def __len__(self):
    return len(self.buffer)

  def append(self, experience):
    self.buffer.append(experience)
  
  def sample(self, batch_size):
    return random.sample(self.buffer, batch_size)

In [ ]:
### Create the RLDataset class, which is used to create a dataset object out of samples from the buffer to be used as input for the training step

class RLDataset(IterableDataset):

  def __init__(self, buffer, sample_size=200):
    self.buffer = buffer
    self.sample_size = sample_size
  
  # This function sequentially gives an experience from the buffer to the pyTorch when requested
  def __iter__(self):
    for experience in self.buffer.sample(self.sample_size):
      yield  experience


In [ ]:
### Create a function to incrementally move the weights of the target network closer to those of the main network

def polyak_average(net, target_net, tau=0.01):
    for qp, tp in zip(net.parameters(), target_net.parameters()):
        tp.data.copy_(tau * qp.data + (1 - tau) * tp.data)

In [ ]:
### Create a function which wraps the default environment from gym in layers of additional funcitonality

def create_environment(name, max_steps):
  env = gym.make(name)
  env = TimeLimit(env, max_episode_steps=max_steps)
  #env = RecordVideo(env, video_folder='./videos', episode_trigger=lambda x: x % 50 == 0)
  env = RecordEpisodeStatistics(env)
  return env

In [ ]:
### This class enacts the TD3 algorithm

class TD3(LightningModule):

  def __init__(self, env_name, capacity=50_000, batch_size=100, 
               actor_lr=1e-3, critic_lr=1e-3, hidden_size=128, gamma=0.995,
               loss_fn=F.smooth_l1_loss, optim=AdamW, eps_start=1.0, eps_end=0.1,
               eps_last_episode=100, samples_per_epoch=1_000, tau=0.004, epsilon=1, max_steps=400, num_layers=1):
    
    super().__init__()

    self.env = create_environment(env_name, max_steps)
    obs_size = self.env.observation_space.shape[0]
    action_dims = self.env.action_space.shape[0]
    max_action = self.env.action_space.high
    min_action = self.env.action_space.low

    self.q_net1 = DQN(hidden_size, obs_size, action_dims, num_layers)
    self.q_net2 = DQN(hidden_size, obs_size, action_dims, num_layers)
    self.actor = Actor(hidden_size, obs_size, action_dims, min_action, max_action, num_layers)

    self.target_q_net1 = copy.deepcopy(self.q_net1)
    self.target_q_net2 = copy.deepcopy(self.q_net2)
    self.target_actor = copy.deepcopy(self.actor)

    self.epsilon = epsilon

    self.buffer = ReplayBuffer(capacity=capacity)

    self.ep_returns = []

    self.max_return = 0
    self.best_actor = copy.deepcopy(self.actor)

    self.save_hyperparameters()

    self.hp_results = None

    # Fill the buffer up with experience (1000 experiences - i.e. self.hparams.samples_per_epoch)
    while len(self.buffer) < self.hparams.samples_per_epoch:
      print(f"{len(self.buffer)} samples in experience buffer. Filling...")
      self.play_episode(epsilon=self.hparams.eps_start)

  # This function plays a single full episode and stores all of the experience in the buffer
  @torch.no_grad()
  def play_episode(self, actor=None, epsilon=0.):
    state = self.env.reset()
    done = False

    while not done:
      if actor:
        action = actor(state, epsilon=epsilon)
      else:
        action = self.env.action_space.sample()
      next_state, reward, done, info = self.env.step(action)
      exp = (state, action, reward, done, next_state)
      self.buffer.append(exp)
      state = next_state


  # This function performs returns a predicted high returning action with noise applied
  def forward(self, x):
    output = self.actor(x)
    return output
  
  # This function configures the optimizers before the process begins
  def configure_optimizers(self):
    q_net_params = itertools.chain(self.q_net1.parameters(), self.q_net2.parameters())
    q_net_optimizer = self.hparams.optim(q_net_params, lr=self.hparams.critic_lr)
    actor_optimizer = self.hparams.optim(self.actor.parameters(), lr=self.hparams.actor_lr)
    return [q_net_optimizer, actor_optimizer]
  
  # This function creates the dataloader, which feeds experience to the training_step() function.
  def train_dataloader(self):
    dataset = RLDataset(self.buffer, self.hparams.samples_per_epoch)
    dataloader = DataLoader(
        dataset=dataset,
        batch_size = self.hparams.batch_size
    )
    return dataloader
  
  # Training step to update the neural networks
  def training_step(self, batch, batch_idx, optimizer_idx):

    states, actions, rewards, dones, next_states = batch
    rewards = rewards.unsqueeze(1)
    dones = dones.unsqueeze(1)

    # Update the target networks
    polyak_average(self.q_net1, self.target_q_net1, tau=self.hparams.tau)
    polyak_average(self.q_net2, self.target_q_net2, tau=self.hparams.tau)
    polyak_average(self.actor, self.target_actor, tau=self.hparams.tau)

    # If the q_net optimizer is selected, update the Q-network
    if optimizer_idx == 0:
      state_action_values1 = self.q_net1(states, actions)
      state_action_values2 = self.q_net2(states, actions)
      next_actions = self.target_actor(next_states, epsilon=self.epsilon, noise_clip=0.05)
      # Select the lowest next state/action value
      next_state_action_values = torch.min(
          self.target_q_net1(next_states, next_actions),
          self.target_q_net2(next_states, next_actions)
      )
      # Set all terminal states to have a value of 0
      next_state_action_values[dones] = 0.0
      expected_state_action_values = rewards + self.hparams.gamma * next_state_action_values

      q_loss1 = self.hparams.loss_fn(state_action_values1, expected_state_action_values)
      q_loss2 = self.hparams.loss_fn(state_action_values2, expected_state_action_values)
      total_loss = q_loss1 + q_loss2
      self.log_dict({"episode/Q-Loss": total_loss})
      return total_loss

    # If the actor optimizer is selected, update the Actor network
    elif optimizer_idx == 1 and batch_idx % 2 == 0:
      mu = self.actor.mu(states)
      actor_loss = - self.q_net1(states, mu).mean()
      self.log_dict({"episode/Actor Loss": actor_loss})
      return actor_loss


  # This function is called at the end of each epoch
  def training_epoch_end(self, training_step_outputs):

    # Decrease epsilon value as time goes on to reduce exploration and increase reward
    self.epsilon = max(
        self.hparams.eps_end,
        self.hparams.eps_start - self.current_epoch / self.hparams.eps_last_episode
    )

    # An episode is played out in full at the end of each epoch, therefore one epoch lasts as long as one episode
    self.play_episode(actor=self.actor, epsilon=self.epsilon)
    self.log('episode/Return', self.env.return_queue[-1])
    

    # If best actor, save as best actor
    if self.env.return_queue[-1] > self.max_return:
      self.best_actor = copy.deepcopy(self.actor)

    # Every 100 episodes calculate and log the hp metric
    if len(self.ep_returns)%100 == 0:
      hp_met = self.calculate_hp_metric()
      self.log('hp_metric', hp_met)

    self.ep_returns.append(self.env.return_queue[-1])


  # This function calculates the hp metric used by Optuna to select the next parameters
  # This metric is the average reward of 10 episodes (tests), and is calculated every 100 episodes of training
  def calculate_hp_metric(self):
    num_tests = 10
    ep_returns = 0
    for test in range(num_tests):
      state = self.env.reset()
      done = False
      while not done:
        action = self.actor(state, epsilon=0)
        next_state, reward, done, info = self.env.step(action)
        exp = (state, action, reward, done, next_state)
        self.buffer.append(exp)
        state = next_state
        ep_returns += reward
    return (ep_returns / num_tests)
    


In [ ]:
### Setting up the tensorboard to display the results

!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

In [ ]:
### Create the objective function for Optuna

def objective(trial):

  # Test hidden_size between 50 and 500
  hidden_size = trial.suggest_int("hidden_size", 50, 500)
  # Test num_layers between 1 and 3
  num_layers = trial.suggest_int("num_layers", 1, 3)

  algo = TD3('BipedalWalker-v3', hidden_size=hidden_size, num_layers=num_layers)

  callback = PyTorchLightningPruningCallback(trial, monitor='hp_metric')

  trainer = Trainer(
      gpus=num_gpus,
      max_epochs=3000,
      # callbacks are functions that are executed after an epoch of training is complete
      callbacks=[callback]
  )

  hyperparameters = dict(hidden_size=hidden_size, num_layers=num_layers)

  # Send info to tensorboard
  trainer.logger.log_hyperparams(hyperparameters)
  # Train the algorithm
  trainer.fit(algo)

  # Store the results of the hp metric
  this_It = pd.DataFrame(algo.ep_returns)
  this_It.columns = [f'It [{hidden_size}, {num_layers}]']
  this_It.to_csv(f'TD3_Optuna_results_[{algo.hparams.hidden_size}, {algo.hparams.num_layers}].csv')

  return trainer.callback_metrics['hp_metric'].item()


In [ ]:
### Create the study for Optuna

# Pruner stops trials early of not very promising
pruner = optuna.pruners.SuccessiveHalvingPruner()
# Tell optuna to maximise the value we are monitoring inside the objective function (ie 'hp_metric')
study = optuna.create_study(direction='maximize', pruner=pruner)

[I 2022-07-17 11:03:27,130] A new study created in memory with name: no-name-dc5a0e63-9632-485a-8512-32599bfac631


In [ ]:
### Running the hyperparameter search

# n_trials is the number of different test combinations used in the search
study.optimize(objective, n_trials=50)

0 samples in experience buffer. Filling...
400 samples in experience buffer. Filling...
527 samples in experience buffer. Filling...
598 samples in experience buffer. Filling...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


998 samples in experience buffer. Filling...



  | Name          | Type  | Params
----------------------------------------
0 | q_net1        | DQN   | 210 K 
1 | q_net2        | DQN   | 210 K 
2 | actor         | Actor | 210 K 
3 | target_q_net1 | DQN   | 210 K 
4 | target_q_net2 | DQN   | 210 K 
5 | target_actor  | Actor | 210 K 
6 | best_actor    | Actor | 210 K 
----------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total params
5.900     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

...

Training: 0it [00:00, ?it/s]

[I 2022-07-18 09:47:51,509] Trial 39 finished with value: 96.78961360647165 and parameters: {'hidden_size': 420, 'num_layers': 2}. Best is trial 30 with value: 130.7904513843903.


0 samples in experience buffer. Filling...
54 samples in experience buffer. Filling...
132 samples in experience buffer. Filling...
212 samples in experience buffer. Filling...
612 samples in experience buffer. Filling...


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type  | Params
----------------------------------------
0 | q_net1        | DQN   | 513 K 
1 | q_net2        | DQN   | 513 K 
2 | actor         | Actor | 513 K 
3 | target_q_net1 | DQN   | 513 K 
4 | target_q_net2 | DQN   | 513 K 
5 | target_actor  | Actor | 513 K 
6 | best_actor    | Actor | 513 K 
----------------------------------------
3.6 M     Trainable params
0         Non-trainable params
3.6 M     Total params
14.385    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
study.best_params